In [1]:
## GeNNus Simone & Paolo

In [6]:
import torch
import tqdm
import os

In [1]:
DATASET_SIZE = "xs"
DATASET_TYPE = "waveform"

DATASET_FOLDER = f"./data/{DATASET_TYPE}"

DATASET_NUM_SAMPLES_PER_SECOND = 8000
DATASET_NUM_CHANNELS = 1

DATASET_NAME = f"fma_{DATASET_SIZE}_resampled_{DATASET_NUM_SAMPLES_PER_SECOND}_rechanneled_{DATASET_NUM_CHANNELS}"

dataset_path = f"{DATASET_FOLDER}/{DATASET_NAME}"

In [2]:
audio_path_list = []

for path, subdirs, files in os.walk(dataset_path):
    for name in files:
        file_audio_path = os.path.join(path, name)
        #print(file_audio_path)

        if name != '.DS_Store':
            audio_path_list.append(file_audio_path)

audio_path_list = sorted(audio_path_list , reverse= True)
# lista di tutti i path     

In [ ]:
audio_path_list

In [ ]:
single_tensors = []
labels = []

for p in audio_path_list:
    single_tensors.append(torch.load(p))
    #print(p)
    labels.append(p.split("/")[-2])

single_tensors # --> waveforms
labels # --> etichette

In [11]:
import numpy as np
import pandas as pd

In [9]:
stacked_single_tensors = torch.cat(single_tensors).numpy()
#stacked_single_tensors.shape
stacked_single_tensors[0].shape

(238000,)

In [22]:
data = pd.DataFrame(stacked_single_tensors)

In [23]:
data.insert(0, "Label", labels, True)

In [24]:
data

,Label,0,1,2,3,4,5,6,7,8,...,237990,237991,237992,237993,237994,237995,237996,237997,237998,237999
0,Rock,0.002090,0.002457,0.003026,-0.003531,0.000593,-0.000422,-0.004120,0.000559,-0.003272,...,0.248962,0.085129,-0.020552,-0.232211,-0.366932,-0.380118,-0.329189,-0.377189,-0.429503,-0.469498
1,Rock,0.000280,0.000392,0.000608,-0.000550,0.000869,0.000239,-0.000428,0.000423,-0.000243,...,-0.054861,-0.078808,-0.092320,-0.119765,-0.017133,-0.037570,-0.003405,-0.004435,-0.020573,-0.000585
2,Rock,-0.001988,-0.003077,-0.002397,0.000297,0.000106,0.001610,0.004233,0.002088,0.001651,...,-0.099118,-0.116422,-0.045899,-0.041989,-0.078948,-0.059464,-0.132240,-0.115517,-0.035373,-0.039236
3,Rock,-0.001400,0.002623,0.003775,0.000649,0.000363,0.004158,0.001411,-0.002852,0.000627,...,0.029565,0.116452,0.115272,0.312533,0.361796,0.340224,0.447560,0.505576,0.706916,0.620015
4,Rock,-0.002345,-0.003766,-0.004752,-0.001465,-0.000347,-0.000928,-0.001434,-0.003766,-0.004756,...,0.062377,-0.057751,-0.148933,-0.111224,-0.028607,0.026423,0.008661,-0.071092,-0.058597,-0.021446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Electronic,-0.012173,0.015278,0.002091,-0.005186,-0.015326,0.007420,-0.012609,-0.090091,-0.095443,...,0.196053,0.266813,0.289611,0.187709,0.149948,0.166143,0.086306,0.032596,-0.081602,-0.092399
86,Electronic,-0.007904,-0.003071,-0.001817,0.000821,0.004140,0.009747,0.005562,0.007040,0.010066,...,0.236072,0.294226,0.274508,0.255579,0.187992,0.121793,0.154414,-0.051746,-0.008415,0.220748
87,Electronic,0.000441,-0.001573,-0.001202,0.000164,-0.004585,-0.004874,-0.001185,0.000745,0.002968,...,-0.078580,-0.023332,-0.046219,0.066519,-0.046962,-0.168407,-0.097263,-0.092109,-0.109068,-0.244396
88,Electronic,0.000145,0.003849,0.001637,0.004717,0.005652,0.005726,0.001288,0.002571,0.001654,...,0.254142,0.372470,0.226492,0.169134,0.197148,0.133427,0.028967,0.041120,-0.026155,0.016971


In [27]:
import random 
def train_valid_test_split( dataset , perc_train , set_seed = 69):

    train = dataset.sample(frac= perc_train,random_state = set_seed )
    test  = dataset.drop(train.index)

    return (train , test )






In [29]:
train_dataset , test_dataset = train_valid_test_split( data , perc_train = .9)

In [58]:
train_dataset.iloc[:,:1]

,Label
31,Jazz
25,Pop
57,Hip-Hop
80,Electronic
52,Hip-Hop
...,...
53,Hip-Hop
88,Electronic
37,Jazz
12,Rock


In [71]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import utils
from tqdm import tqdm


X = train_dataset.iloc[: , 1:]
y = train_dataset.iloc[: , :1]

#Implementing cross validation

k = 3
kf = KFold( n_splits=k, random_state= None , shuffle= True )
model = LogisticRegression()

acc_score = []
for j in range(k):
    x_train , x_validation = train_valid_test_split( X , perc_train = .78 , set_seed= j)
    y_train , y_validation = train_valid_test_split( y , perc_train = .78 , set_seed= j)
    
    model.fit(x_train , y_train)
    pred_values = model.predict(x_validation)
    
    acc = accuracy_score( pred_values , y_validation)
    acc_score.append(acc)

avg_acc_score = sum(acc_score)/k

/Users/paolo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paolo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

In [72]:
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.2222222222222222, 0.3333333333333333, 0.16666666666666666]
Avg accuracy : 0.24074074074074073


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split( stacked_single_tensors , labels, test_size=0.2, random_state=42 )
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(X_train, Y_train)


In [115]:
LogisticPredictions = logistic.predict( X_test )
Logistic_Accuracy = logistic.score( X_test , Y_test) 

In [ ]:
LogisticPredictions

In [118]:
Logistic_Accuracy

0.1111111111111111

In [119]:
from sklearn.metrics import confusion_matrix
confusion_matrix( Y_test , LogisticPredictions , labels = None, sample_weight=None, normalize=None)

array([[0, 4, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0],
       [0, 4, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0],
       [0, 4, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0]])

In [ ]:
from sklearn.model_selection import KFold
key_fold = KFold( n_splits=4 ) 
key_fold.get_n_splits(X_train)

In [79]:
import pandas as pd
df = pd.DataFrame(stacked_single_tensors)
X = df.iloc[:,:-1]
y = labels


In [ ]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import utils
from tqdm import tqdm
set.seed(69)


X = df.iloc[:,:-1]

#convert y values to categorical values

y = pd.DataFrame(labels)

#Implementing cross validation

k = 3
kf = KFold(n_splits=k, random_state= None , shuffle= True )
model = LogisticRegression()

acc_score = []

for train_index , test_index in tqdm( kf.split(X) ):

    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y.iloc[train_index] , y.iloc[test_index]


    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score( pred_values , y_test)
    acc_score.append(acc)

avg_acc_score = sum(acc_score)/k


accuracy of each fold - [0.14814814814814814, 0.1111111111111111, 0.25925925925925924]
Avg accuracy : 0.1728395061728395
